# Fix pathing

In [1]:
import sys


sys.path.append("../..")


In [2]:
import constants

import os


constants.PROJECT_DIRECTORY_PATH = os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(constants.PROJECT_DIRECTORY_PATH))))


# Imports

In [3]:
import plotter
import datahandler

import matplotlib.pyplot
import numpy as np
import pandas as pd
import seaborn as sns
import IPython.display


# Constants

In [84]:
FOLDER_NAME = "mv_stress_test_rds_most_NONE"

FOLDER_PATH = os.path.join(os.path.dirname(constants.PROJECT_DIRECTORY_PATH), "Simulator", "data", FOLDER_NAME)


# Methods

In [85]:
def load_csv(fileName):
    df = pd.read_csv(os.path.join(FOLDER_PATH, fileName + ".csv"))

    response_time_cols = [
        'duration_incident_creation',
        'duration_resource_appointment',
        'duration_resource_preparing_departure',
        'duration_dispatching_to_scene'
    ]
    df['total_response_time'] = df[response_time_cols].sum(axis=1)

    return df


In [86]:
def print_results(df: pd.DataFrame):
    # Define the criteria for response times
    criteria = {
        ('A', True): 12 * 60,
        ('A', False): 25 * 60,
        ('H', True): 30 * 60,
        ('H', False): 40 * 60
    }

    # Function to calculate compliance for each group
    def calculate_compliance(group, triage, urban):
        limit = criteria.get((triage, urban))
        if limit is not None:
            return (group['total_response_time'] < limit).mean()
        return None

    # Calculate statistics and compliance for each group
    results = []
    for (triage, urban), group in df.groupby(['triage_impression_during_call', 'urban']):
        mean = group['total_response_time'].mean()
        median = group['total_response_time'].median()
        compliance = calculate_compliance(group, triage, urban)
        results.append({
            'Triage': triage,
            'Urban': urban,
            'Mean (sec)': mean,
            'Median (sec)': median,
            'Compliance': compliance
        })

    stats = pd.DataFrame(results)

    # Convert mean and median to minutes
    stats['Mean (min)'] = (stats['Mean (sec)'] / 60)
    stats['Median (min)'] = (stats['Median (sec)'] / 60)
    stats.drop(columns=['Mean (sec)', 'Median (sec)'], inplace=True)

    # Map urban values to Yes/No
    stats['Urban'] = stats['Urban'].map({True: 'Yes', False: 'No'})
    
    # Sort values
    stats.sort_values(by=["Urban", "Triage"], ascending=[False, True], inplace=True)
    
    # Display the DataFrame
    formatted_stats = stats.style.format({
        'Mean (min)': "{:.2f}",
        'Median (min)': "{:.2f}",
        'Compliance': "{:.2%}"
    }).hide(axis='index')
    IPython.display.display(formatted_stats)


In [87]:
def table_results():
    def calculate_compliance(df, urban_filter=None):
        if df is None:
            return None

        # Criteria for both triage types in both urban and rural settings
        criteria = {
            ('A', True): 12 * 60,  # 12 minutes for triage 'A' in urban areas
            ('A', False): 25 * 60,  # 25 minutes for triage 'A' in rural areas
            ('H', True): 30 * 60,  # 30 minutes for triage 'H' in urban areas
            ('H', False): 40 * 60  # 40 minutes for triage 'H' in rural areas
        }

        # Initialize counters
        total_compliant_cases = 0
        total_cases = 0

        # Check necessary columns exist
        for urban in [True, False]:
            if urban_filter is not None and urban != urban_filter:
                continue
            for triage in ['A', 'H']:
                filtered_df = df[(df['triage_impression_during_call'] == triage) & (df['urban'] == urban)]
                limit = criteria.get((triage, urban))
                if not filtered_df.empty:
                    # Count compliant cases for this triage type
                    compliant_cases = filtered_df['total_response_time'] < limit
                    total_compliant_cases += compliant_cases.sum()
                    total_cases += len(filtered_df)

        if total_cases > 0:
            # Calculate overall compliance rate across both triage types
            overall_compliance = total_compliant_cases / total_cases
            return overall_compliance
        else:
            return None  # Return None if there are no cases to evaluate

    # Factors to test
    factors = [0.5, 1.0, 1.5]

    results = []

    for factor in factors:
        filename = f"events_numIncidentsFactor={factor:.6f}"
        df = load_csv(filename)
        results.append({
            "Factor": factor,
            "Compliance": calculate_compliance(df),
            "Urban": calculate_compliance(df, urban_filter=True),
            "Rural": calculate_compliance(df, urban_filter=False)
        })

    # Create DataFrame
    results_df = pd.DataFrame(results)

    results_df = results_df.applymap(lambda x: round(x, 4) if isinstance(x, float) else x)

    return results_df



# Main

In [88]:
table_results()


C:\Users\sindr\AppData\Local\Temp\ipykernel_11992\1394679720.py:56: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  results_df = results_df.applymap(lambda x: round(x, 4) if isinstance(x, float) else x)


,Factor,Compliance,Urban,Rural
0,0.5,0.2414,0.2267,0.3333
1,1.0,0.2623,0.2562,0.3043
2,1.5,0.1734,0.1811,0.1071


In [89]:
print_results(load_csv("events_numIncidentsFactor=0.500000"))


Triage,Urban,Compliance,Mean (min),Median (min)
A,Yes,8.33%,26.03,25.90
H,Yes,35.90%,39.31,32.05
V1,Yes,nan%,131.22,111.86
A,No,16.67%,45.91,45.16
H,No,50.00%,41.49,41.57
V1,No,nan%,115.40,124.10


In [90]:
print_results(load_csv("events_numIncidentsFactor=1.000000"))


Triage,Urban,Compliance,Mean (min),Median (min)
A,Yes,12.16%,25.03,21.55
H,Yes,37.21%,40.04,36.83
V1,Yes,nan%,112.08,97.85
A,No,20.00%,36.30,34.75
H,No,50.00%,48.85,47.48
V1,No,nan%,138.82,134.63


In [91]:
print_results(load_csv("events_numIncidentsFactor=1.500000"))


Triage,Urban,Compliance,Mean (min),Median (min)
A,Yes,9.43%,66.15,27.68
H,Yes,24.82%,92.53,49.78
V1,Yes,nan%,178.35,115.66
A,No,5.88%,47.30,37.68
H,No,18.18%,182.38,55.95
V1,No,nan%,160.15,154.19
